In [ ]:
import sys, json

sys.path.insert(0, "../")

from lib import *

In [ ]:
user_input = {
    "config_file": "vd_1x8x14/vd_1x8x14_config",
    "root_file": ["wbkg"],
    "rewrite": True,
    "debug": True,
}
info = json.load(open("../config/" + user_input["config_file"] + ".json", "r"))
data_filter = {
    "max_energy": 20,
    "min_energy": 0,
    "pre_nhits": 3,
    "primary": True,
    "neutron": True,
}
true, reco, filter_idx = compute_root_workflow(
    user_input, info, data_filter, workflow="SMEARING", debug=user_input["debug"]
)
rprint(
    "-> Found %i electron candidates out of %i events!"
    % (len(filter_idx), reco["Event"].size)
)
run = {"Truth": true, "Reco": reco}

In [ ]:
dict_array = []

info["DEFAULT_ANALYSIS_ENERGY"] = "TotalEnergy"

config = user_input["config_file"].split("/")[0]
df_dict, df_dict[config] = {}, {}

analysis_info = json.load(open("../import/analysis.json", "r"))
energy_edges = np.linspace(
    analysis_info["RECO_ENERGY_RANGE"][0],
    analysis_info["RECO_ENERGY_RANGE"][1],
    analysis_info["RECO_ENERGY_BINS"] + 1,
)
energy_centers = (energy_edges[1:] + energy_edges[:-1]) / 2
bin_width = energy_edges[1] - energy_edges[0]

# Get the list of generators for this file
for jdx, name in enumerate(user_input["root_file"]):
    truth_filter = np.asarray(run["Truth"]["Name"]) == name
    reco_filter = np.asarray(run["Reco"]["Name"]) == name
    gen_list = np.unique(run["Reco"]["Generator"][reco_filter])
    print("\n- Generators found for", name, ":", gen_list, "\n")
    # Get the total number of events for this file

    int_time = len(run["Truth"]["Event"]) * info["TIMEWINDOW"]
    # print("Total number of events for", name, "is", str(count_truth_df["Marley"]))
    
    # Start generator loop
    for kdx, gen in enumerate(gen_list):
        filters = compute_solarnuana_filters(
            run = run,
            configs = {config: config + "_config"},
            config = config,
            name = name,
            gen = gen,
            filter_list=["Primary","NHits"],
            params={"PRESELECTION_NHITS":3},
            cummulative=True,
            debug=False,
        )

        gen_label = get_gen_label({config: config + "_config"})[
            (info["GEOMETRY"], info["VERSION"], gen)
        ]
        this_dict_array, df_dict[config][gen_label] = compute_solar_spectrum(
            run=run,
            info=info,
            configs={config: config + "_config"},
            config=config,
            names={config: user_input["root_file"]},
            name=name,
            gen=gen,
            energy_edges=energy_edges,
            int_time=int_time,
            filters=filters,
            truth_filter=truth_filter,
            reco_filter=reco_filter,
            input_dm2=[7.4e-5],
            input_sin13=[0.021],
            input_sin12=[0.303],
            auto=False,
            save=False,
            debug=False,
        )

        dict_array = dict_array + this_dict_array

In [ ]:
solar_df = pd.DataFrame(dict_array)
# display(solar_df)
filter_names = solar_df[ 
    (solar_df['Filter'] == "All")
    # + (solar_df['Filter'] == "All+Primary") 
    + (solar_df['Filter'] == "All+NHits") 
    # + (solar_df['Filter'] == "All+Fiducial")
    # + (solar_df['Filter'] == "All+MainClEnergy")
    # + (solar_df['Filter'] == "All+OpFlash")
    ].index
solar_df = solar_df.drop(index=filter_names, inplace = False)
plot_df = solar_df.explode(["TotalCounts", "Energy", "Efficiency"])

plot_df["SimpleName"] = plot_df["GenLabel"].map(
    get_simple_name(plot_df["GenLabel"].unique())
)
this_plot_df = plot_df[(plot_df["TotalCounts"] > 0)]
# display(this_plot_df[this_plot_df["Energy"] > 0].groupby(["Name","Version","Filter","Generator"])["TotalCounts"].sum())
fig = px.bar(
    data_frame=this_plot_df,
    x=this_plot_df["Energy"]+2,
    y="TotalCounts",
    facet_col="Version",
    facet_row="Filter",
    facet_row_spacing=0.1,
    color="GenLabel",
    # line_shape = "hvh",
    color_discrete_sequence=pd.Series(this_plot_df["GenLabel"].unique()).map(
        get_bkg_color(this_plot_df["GenLabel"].unique())
    ),
    barmode="overlay",
)

# fig.update_layout(xaxis1_title="Reco Energy [MeV]",xaxis_title="Reco Energy [MeV]",yaxis_title="Counts [Hz]",yaxis1_title="Counts [Hz]")
fig = format_coustom_plotly(
    fig,
    log=(False, True),
    figsize=(1200, 800),
    fontsize=18,
    ranges=([0, 25], [0, 6]),
    tickformat=(",.0f", ".0s"),
    add_units=True,
)
fig.update_yaxes(title="Events/400kt-yr")
fig.update_xaxes(title="Reco Energy [MeV]")
fig.update_layout(bargap=0)

fig = plot_detected_solar_spectrum(fig,0,np.arange(0,25,0.5),mass=10e9*8*60*60*24*365)

fig.show()

solar_df.groupby("GenLabel")["TotalCounts"].sum()

In [ ]:
plot_df = pd.DataFrame(dict_array)
plot_df = explode(plot_df,["Efficiency","Energy","TotalCounts"])
# Change 0 to NaN for plotting
# display(plot_df)
this_plot_df = plot_df[(plot_df["Generator"] == 1)]
fig = px.line(
    this_plot_df,
    x="Energy",
    y="Efficiency",
    color="Filter",
    facet_col="Version",
    line_shape = "hvh",
    # category_orders={"Version":["vd_refactored"]},
    # log_y=True,
    # trendline="lowess",
    )

fig = format_coustom_plotly(fig,tickformat=(".2i",".2i"),figsize=(800,600),add_units=True)
fig.update_traces(marker=dict(size=10,line=dict()))
fig.show()

In [ ]:
gen_label = "Marley"
this_df = df_dict[list(df_dict.keys())[0]][gen_label][list(df_dict[list(df_dict.keys())[0]][gen_label].keys())[0]]
fig = px.imshow(this_df,
    origin="lower",
    aspect="auto",
    color_continuous_scale="turbo",
    labels=dict(y="Nadir Angle cos("+unicode("eta")+")", x="RecoEnergy [MeV]"),
)
fig = format_coustom_plotly(fig,figsize=(800,600))
fig.update_layout(coloraxis_colorbar=dict(title="Events/400kt-yr"))
fig.update_layout(title="Event Count Distribution")
fig.show()
print("Total number of events %.2f"%this_df.sum().sum())